In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.

In [1]:
import pandas as pd
import re, math
import numpy as np
import pandas as pd
from scipy.special import softmax

from simpletransformers.ner import NERModel
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [2]:
columns = ['Name', 'Street', 'SubArea', 'Area', 'City']

In [3]:
def generate_data(file_name):
    data = pd.read_csv(file_name)
    count = 0
    labels = set()

    def get_value(x):
        nonlocal count
        results = []
        address = x["address"]  # Ensure this column exists or is constructed properly
        previous = ""
        _temp = x.to_dict()
        for each in address.split():
            match = False
            clean_word = each.strip(" ,.-")
            for col in columns:
                if clean_word in str(_temp[col]):
                    if previous == col:
                        results.append((count, each, "I-"+col))
                    else:
                        results.append((count, each, "B-"+col))
                    previous = col
                    match = True
                    break
            if not match:
                results.append((count, each, "O"))
            labels.add(results[-1][2])  # Capture label format
        count += 1
        return results

    values = data.apply(get_value, axis=1)
    flat_list = [item for sublist in values for item in sublist]
    train_df = pd.DataFrame(flat_list, columns=["sentence_id", "words", "labels"])
    return train_df, labels

In [4]:
train_df, train_labels = generate_data("/content/normalized.csv")
test_df, test_labels = generate_data("/content/val.csv")

In [5]:
# Save the DataFrame in a suitable format
train_df.to_csv(r'pandas1.txt', header=None, index=None, sep=' ', mode='a')

In [7]:
# Model arguments
arguments = {
    "save_eval_checkpoints": False,
    "save_steps": -1,
    'overwrite_output_dir': True,
    "save_model_every_epoch": False,
    'reprocess_input_data': True,
    "train_batch_size": 8,
    'num_train_epochs': 40,
    "max_seq_length": 256,
    "gradient_accumulation_steps": 8,
    "learning_rate": 4e-5,
    "do_lower_case": False,
    "adam_epsilon": 1e-8,
    "eval_batch_size": 8,
    "fp16": True,
    'use_cuda': False
}

In [8]:
model = NERModel("distilbert", "distilbert-base-uncased", args=arguments, labels=list(train_labels), use_cuda=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
model.train_model(train_df)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

Running Epoch 1 of 40:   0%|          | 0/36 [00:00<?, ?it/s]

Running Epoch 2 of 40:   0%|          | 0/36 [00:00<?, ?it/s]

...

Running Epoch 37 of 40:   0%|          | 0/36 [00:00<?, ?it/s]

Running Epoch 38 of 40:   0%|          | 0/36 [00:00<?, ?it/s]

Running Epoch 39 of 40:   0%|          | 0/36 [00:00<?, ?it/s]

Running Epoch 40 of 40:   0%|          | 0/36 [00:00<?, ?it/s]

(160, 0.5828358312719502)

In [12]:
labels =list(train_labels)

In [13]:
labels

['B-City',
 'B-Name',
 'B-Street',
 'I-City',
 'I-SubArea',
 'O',
 'B-SubArea',
 'B-Area',
 'I-Area',
 'I-Name',
 'I-Street']

In [18]:
result, model_outputs, predictions = model.eval_model(train_df)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/36 [00:00<?, ?it/s]

In [19]:
result

{'eval_loss': 0.06573496947789358,
 'precision': 0.9724454649827784,
 'recall': 0.966343411294923,
 'f1_score': 0.9693848354792561}

In [20]:
result, model_outputs, predictions = model.eval_model(test_df)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
result #test-results

{'eval_loss': 2.0402033030986786,
 'precision': 0.25735294117647056,
 'recall': 0.25547445255474455,
 'f1_score': 0.2564102564102564}

In [23]:
import pandas as pd

val_data = pd.read_csv("/content/val.csv")
addresses = val_data['address'].tolist()

In [24]:
predictions, raw_outputs = model.predict(addresses)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/4 [00:00<?, ?it/s]

In [26]:
for i in range(min(5, len(predictions))):
    print(f"Address {i+1}: {addresses[i]}")
    print("Predictions:")
    print(predictions[i])
    print("\n")

Address 1: 4rth Floor bahria complex 4 main khaliq uz zaman road gizri clifton 4rth Floor bahria complex 4 main khaliq uz zaman road gizri clifton
Predictions:
[{'4rth': 'B-Name'}, {'Floor': 'I-Name'}, {'bahria': 'O'}, {'complex': 'B-SubArea'}, {'4': 'I-SubArea'}, {'main': 'B-Street'}, {'khaliq': 'B-Street'}, {'uz': 'I-Street'}, {'zaman': 'I-Street'}, {'road': 'I-Street'}, {'gizri': 'O'}, {'clifton': 'B-Area'}, {'4rth': 'O'}, {'Floor': 'I-Name'}, {'bahria': 'O'}, {'complex': 'B-SubArea'}, {'4': 'I-SubArea'}, {'main': 'B-Street'}, {'khaliq': 'B-Street'}, {'uz': 'I-Street'}, {'zaman': 'I-Street'}, {'road': 'I-Street'}, {'gizri': 'B-Area'}, {'clifton': 'B-Area'}]


Address 2: Flat f74 block f Noman avenue main rashid mehnas road johar block 20 karachi Noman avenu
Predictions:
[{'Flat': 'B-Name'}, {'f74': 'I-Name'}, {'block': 'B-SubArea'}, {'f': 'O'}, {'Noman': 'B-Street'}, {'avenue': 'B-Street'}, {'main': 'I-Street'}, {'rashid': 'I-Street'}, {'mehnas': 'I-Street'}, {'road': 'I-Street'}, {

In [29]:
for i, address in enumerate(addresses[:5]):
    print(f"Address {i+1}: {address}")
    print("Predictions:")
    for prediction_dict in predictions[i]:
        for token, label in prediction_dict.items():
            print(f"  {token}: {label}")
    print("\n")


Address 1: 4rth Floor bahria complex 4 main khaliq uz zaman road gizri clifton 4rth Floor bahria complex 4 main khaliq uz zaman road gizri clifton
Predictions:
  4rth: B-Name
  Floor: I-Name
  bahria: O
  complex: B-SubArea
  4: I-SubArea
  main: B-Street
  khaliq: B-Street
  uz: I-Street
  zaman: I-Street
  road: I-Street
  gizri: O
  clifton: B-Area
  4rth: O
  Floor: I-Name
  bahria: O
  complex: B-SubArea
  4: I-SubArea
  main: B-Street
  khaliq: B-Street
  uz: I-Street
  zaman: I-Street
  road: I-Street
  gizri: B-Area
  clifton: B-Area


Address 2: Flat f74 block f Noman avenue main rashid mehnas road johar block 20 karachi Noman avenu
Predictions:
  Flat: B-Name
  f74: I-Name
  block: B-SubArea
  f: O
  Noman: B-Street
  avenue: B-Street
  main: I-Street
  rashid: I-Street
  mehnas: I-Street
  road: I-Street
  johar: I-Street
  block: B-SubArea
  20: B-Name
  karachi: B-City
  Noman: O
  avenu: O


Address 3: House no 23 street 9 sector 4 haroon bahria naval colony hub river roa

In [ ]:
import csv

with open('/content/predicted_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Word", "Predicted Label"])

    for address_predictions in predictions:
        for pred in address_predictions:
            writer.writerow([pred['word'], pred['entity']])